In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile
import yaml

from loguru import logger

import numpy as np
import lightning
import torch
import datamol as dm
import pandas as pd

import graphium

Using backend: pytorch


In [9]:
import graphium

dataset_dir = "/home/hadim/test-data"
data_path = graphium.data.utils.download_graphium_dataset("graphium-zinc-micro", output_path=dataset_dir)
print(data_path)

/home/hadim/test-data/graphium-zinc-micro


In [8]:
data_path

'/home/hadim/test-data/graphium-zinc-micro'